In [1]:
import pandas as pd
import re

import past_guest_module as pm

import sys
sys.path.append('../zoho_data_app/')

import main_module as m

%load_ext autoreload
%autoreload 2


In [2]:
path = r'C:\Users\fajar\Documents\Python\Data\check_country_code_analytic.xlsx'
df_analytic = pd.read_excel(path, dtype='string', sheet_name='Sheet1')
count_data_awal = df_analytic.shape[0]
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK068763,Karma Resorts,Other,Past Guests,Karma Kandara,PF301495,Mr,Andy,Lim,Male,...,2025-01-31 00:00:00,5,HOUSE_USE,V_OWNER_NRB,COMP,2,0,0,Email and Phone,NO
1,KK068421,Karma Resorts,Other,Past Guests,Karma Kandara,PF300973,Mr,Ben,Tutton,Male,...,2025-01-26 00:00:00,4,HOUSE_USE,V_OWNER_NRB,COMP,3,1,0,Email and Phone,NO
2,KB047057,Karma Resorts,Other,Past Guests,Karma Bavaria,PF301214,Mr,Daniel,Markovich,Male,...,2025-01-28 00:00:00,6,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
3,KK059478,Karma Resorts,Other,Past Guests,Karma Kandara,PF282865,Mr,Sol,Michael,Male,...,2025-01-29 00:00:00,6,HOUSE_USE,PRESS,MEDIA,4,0,0,Email and Phone,NO
4,KK068044,Karma Resorts,Other,Past Guests,Karma Kandara,PF220364,Mr,Harry,Mcasey,Male,...,2025-01-29 00:00:00,4,HOUSE_USE,V_OWNER_NRB,COMP,4,4,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,KB044258,Karma Resorts,Other,Past Guests,Karma Bavaria,<NA>,<NA>,Viscotec Pumpen- Und Dosiertechnik,Gmbh,Unknown,...,2025-02-03 18:00:00,0,CB,CB,GROUP,1,0,0,Email Only,NO
109,KK067739,Karma Resorts,Other,Past Guests,Karma Kandara,<NA>,<NA>,<NA>,Zhu Xiaorong,Unknown,...,2025-02-01 00:00:00,2,OTA,OTA,RETAIL,4,0,0,Email Only,NO
110,KB045329,Karma Resorts,Other,Past Guests,Karma Bavaria,<NA>,Firma,Mice,Service,Unknown,...,2025-01-31 15:00:00,0,CB,CB,GROUP,1,0,0,Email Only,NO
111,KB043376,Karma Resorts,Other,Past Guests,Karma Bavaria,<NA>,<NA>,Munich Re,T30945310,Unknown,...,2025-01-30 14:00:00,0,CB,CB,GROUP,1,0,0,Email Only,NO


In [3]:
df_analytic.columns

Index(['Booking_Id', 'Lead Brand', 'Lead Sub- Brand', 'Lead Source',
       'Resort_Name', 'ProfileRef', 'Salutation', 'First_Name', 'Surname',
       'Gender', 'Language', 'DOB', 'Street', 'Area', 'Town', 'County',
       'Country', 'Nationality', 'PostCode', 'TelNo', 'MobileNo', 'Email',
       'Exclude From Mailing', 'Opt In', 'Booking Status', 'DateArrival',
       'DateDepart', 'No. of Nights', 'Market_Segment', 'Media_Code',
       'Media Code Group', 'No. of Adult', 'No. of Children', 'No. of Infants',
       'Contact Type', 'Odyssey Members'],
      dtype='object')

In [4]:
df_analytic.fillna('empty', inplace=True)

In [5]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


In [6]:
m.clean_space(df_email, 'Email')
df_email['Email'] = m.lowercase(df_email, 'Email')

c:\Users\fajar\Documents\Python\Task Offline\past_guest_toni_app\../zoho_data_app\main_module.py:90: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)


In [7]:
# hitung jumlah duplicate
count_duplicate_general = df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])].shape[0]

df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])]

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
26,KB047186,Karma Resorts,Other,Past Guests,Karma Bavaria,PF298112,Frau,Alexandra,Babic,Female,...,2025-02-02 00:00:00,1,COMP,GER_OWE,COMP,3,0,2,Email and Phone,NO
33,KB046214,Karma Resorts,Other,Past Guests,Karma Bavaria,PF213986,empty,Sabine,Reiner,Unknown,...,2025-02-02 00:00:00,2,COMP,GER_OWE,COMP,3,0,0,Email and Phone,NO
41,KB046155,Karma Resorts,Other,Past Guests,Karma Bavaria,PF295139,Firma,Karma,Bavaria,Unknown,...,2025-01-26 14:30:00,0,LGR,LGR,GROUP,1,0,0,Email Only,NO
44,KK067284,Karma Resorts,Other,Past Guests,Karma Kandara,PF297585,empty,Shiju,Shaji,Unknown,...,2025-02-03 00:00:00,1,COMP,RESERVED,KC,1,0,0,Email and Phone,NO
45,KK067144,Karma Resorts,Other,Past Guests,Karma Kandara,PF297585,empty,Shiju,Shaji,Unknown,...,2025-02-03 00:00:00,1,COMP,RESERVED,KC,1,0,0,Email and Phone,NO
46,KK067283,Karma Resorts,Other,Past Guests,Karma Kandara,PF297585,empty,Shiju,Shaji,Unknown,...,2025-02-03 00:00:00,1,COMP,RESERVED,KC,1,0,0,Email and Phone,NO
47,KK067027,Karma Resorts,Other,Past Guests,Karma Kandara,PF297585,empty,Shiju,Shaji,Unknown,...,2025-02-03 00:00:00,1,COMP,RESERVED,KC,1,0,0,Email and Phone,NO
48,KK067149,Karma Resorts,Other,Past Guests,Karma Kandara,PF297585,empty,Shiju,Shaji,Unknown,...,2025-02-03 00:00:00,1,COMP,RESERVED,KC,1,0,0,Email and Phone,NO
49,KK067236,Karma Resorts,Other,Past Guests,Karma Kandara,PF297585,empty,Shiju,Shaji,Unknown,...,2025-02-03 00:00:00,1,COMP,RESERVED,KC,1,0,0,Email and Phone,NO
50,KK067235,Karma Resorts,Other,Past Guests,Karma Kandara,PF297585,empty,Shiju,Shaji,Unknown,...,2025-02-03 00:00:00,1,COMP,RESERVED,KC,1,0,0,Email and Phone,NO


In [8]:
df_analytic.drop_duplicates(subset=['TelNo','MobileNo','Email'], inplace=True)
df_analytic.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = df_analytic.shape[0]

df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK068763,Karma Resorts,Other,Past Guests,Karma Kandara,PF301495,Mr,Andy,Lim,Male,...,2025-01-31 00:00:00,5,HOUSE_USE,V_OWNER_NRB,COMP,2,0,0,Email and Phone,NO
1,KK068421,Karma Resorts,Other,Past Guests,Karma Kandara,PF300973,Mr,Ben,Tutton,Male,...,2025-01-26 00:00:00,4,HOUSE_USE,V_OWNER_NRB,COMP,3,1,0,Email and Phone,NO
2,KB047057,Karma Resorts,Other,Past Guests,Karma Bavaria,PF301214,Mr,Daniel,Markovich,Male,...,2025-01-28 00:00:00,6,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
3,KK059478,Karma Resorts,Other,Past Guests,Karma Kandara,PF282865,Mr,Sol,Michael,Male,...,2025-01-29 00:00:00,6,HOUSE_USE,PRESS,MEDIA,4,0,0,Email and Phone,NO
4,KK068044,Karma Resorts,Other,Past Guests,Karma Kandara,PF220364,Mr,Harry,Mcasey,Male,...,2025-01-29 00:00:00,4,HOUSE_USE,V_OWNER_NRB,COMP,4,4,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,KLM007720,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF297476,empty,Emma Jane,Martin,Unknown,...,2025-02-02 00:00:00,1,OTA,OTA,RETAIL,2,1,0,Email and Phone,NO
67,KLM007662,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF296748,empty,Russell,Martin,Unknown,...,2025-02-02 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
68,KSH020271,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF287910,empty,Peter,Kenyon,Unknown,...,2025-01-31 00:00:00,4,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
69,KLM008155,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF301640,Mrs,Nicola,Peters,Female,...,2025-01-26 00:00:00,1,DIR,TEL,RETAIL,2,0,0,Email and Phone,NO


In [9]:
find_member_booking = df_analytic.copy()

find_member_booking.loc[find_member_booking['TelNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['MobileNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'
find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

find_member_booking.replace('empty', '', inplace=True)

In [10]:
find_member_booking.to_csv(r'C:\Users\fajar\Documents\Python\Data\cek_member_zoho_analytic.csv', index=False)

In [11]:
count_dedup_member = df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])].shape[0]


df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])]


,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK068763,Karma Resorts,Other,Past Guests,Karma Kandara,PF301495,Mr,Andy,Lim,Male,...,2025-01-31 00:00:00,5,HOUSE_USE,V_OWNER_NRB,COMP,2,0,0,Email and Phone,NO
20,KB045753,Karma Resorts,Other,Past Guests,Karma Bavaria,PF140933,Frau,Claudia,Göbl,Female,...,2025-01-26 00:00:00,2,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
23,KB045799,Karma Resorts,Other,Past Guests,Karma Bavaria,PF192683,Frau,Kathrin,Haas,Female,...,2025-01-26 00:00:00,2,COMP,GER_OWE,COMP,2,0,0,Email Only,NO
25,KB047184,Karma Resorts,Other,Past Guests,Karma Bavaria,PF298112,Frau,Alexandra,Babic,Female,...,2025-02-02 00:00:00,3,COMP,GER_OWE,COMP,2,0,2,Email and Phone,NO
28,KB045613,Karma Resorts,Other,Past Guests,Karma Bavaria,PF065347,Herr,Dominik,Peckmann,Female,...,2025-01-26 00:00:00,2,COMP,GER_OWE,COMP,3,1,0,Email and Phone,NO
31,KB045677,Karma Resorts,Other,Past Guests,Karma Bavaria,PF213986,empty,Sabine,Reiner,Unknown,...,2025-01-26 00:00:00,2,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
39,KK067439,Karma Resorts,Other,Past Guests,Karma Kandara,PF298926,Mr,Abc,Def,Male,...,2025-02-01 00:00:00,3,DIGITAL,GHPA,DIGITAL,2,0,0,Email and Phone,NO
40,KK067146,Karma Resorts,Other,Past Guests,Karma Kandara,PF297585,empty,Shiju,Shaji,Unknown,...,2025-02-03 00:00:00,1,COMP,RESERVED,KC,2,0,0,Email and Phone,NO


In [12]:
df_analytic.drop(df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])
].index, inplace=True)

df_analytic.reset_index(drop=True, inplace=True)

count_after_dedup_member = df_analytic.shape[0]

In [13]:
count_data_akhir = df_analytic.shape[0]

In [14]:
df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK068421,Karma Resorts,Other,Past Guests,Karma Kandara,PF300973,Mr,Ben,Tutton,Male,...,2025-01-26 00:00:00,4,HOUSE_USE,V_OWNER_NRB,COMP,3,1,0,Email and Phone,NO
1,KB045929,Karma Resorts,Other,Past Guests,Karma Bavaria,PF294178,Herr,Frank,Kremin,Male,...,2025-01-26 00:00:00,2,DIR,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO
2,KB044743,Karma Resorts,Other,Past Guests,Karma Bavaria,PF287916,Dr,Miriam,Nagel,Female,...,2025-01-26 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
3,KB045003,Karma Resorts,Other,Past Guests,Karma Bavaria,PF289135,empty,Birgit,Mederer,Female,...,2025-01-26 00:00:00,2,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
4,KB047154,Karma Resorts,Other,Past Guests,Karma Bavaria,PF300062,Frau,Mariana,Kachala,Female,...,2025-01-26 00:00:00,2,OTA,OTA,RETAIL,3,3,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,KLM007720,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF297476,empty,Emma Jane,Martin,Unknown,...,2025-02-02 00:00:00,1,OTA,OTA,RETAIL,2,1,0,Email and Phone,NO
59,KLM007662,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF296748,empty,Russell,Martin,Unknown,...,2025-02-02 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
60,KK066319,Karma Resorts,Other,Past Guests,Karma Kandara,PF297125,Ms,Sala,Buna,Unknown,...,2025-02-02 00:00:00,10,DIGITAL,GHPA,DIGITAL,2,1,0,Email and Phone,NO
61,KLM006630,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF285805,Dr,Fiona,Rae,Unknown,...,2025-02-02 00:00:00,3,DIR,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO


In [15]:
df_analytic['DOB'].loc[df_analytic['DOB']=='']

Series([], Name: DOB, dtype: string)

In [16]:
df_analytic['DOB'].replace('empty', '', inplace=True)
dob_con_date = pd.to_datetime(df_analytic['DOB'], errors='coerce')
dob_date = dob_con_date.dt.strftime('%Y-%m-%d')

arr_con_date = pd.to_datetime(df_analytic['DateArrival'])
arr_date = arr_con_date.dt.strftime('%Y-%m-%d')

dep_con_date = pd.to_datetime(df_analytic['DateDepart'])
dep_date = dep_con_date.dt.strftime('%Y-%m-%d')

df_analytic['DOB'] = dob_date
df_analytic['DateArrival'] = arr_date
df_analytic['DateDepart'] = dep_date


C:\Users\fajar\AppData\Local\Temp\ipykernel_19216\1513714307.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_analytic['DOB'].replace('empty', '', inplace=True)


In [17]:
df_analytic.replace('empty', '', inplace=True)

df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK068421,Karma Resorts,Other,Past Guests,Karma Kandara,PF300973,Mr,Ben,Tutton,Male,...,2025-01-26,4,HOUSE_USE,V_OWNER_NRB,COMP,3,1,0,Email and Phone,NO
1,KB040433,Karma Resorts,Other,Past Guests,Karma Bavaria,PF080008,,Kea,Marketing Gmbh,Unknown,...,2025-01-26,0,LGR,LGR,GROUP,1,0,0,Email Only,NO
2,KB045098,Karma Resorts,Other,Past Guests,Karma Bavaria,PF289645,Frau,Kamila,Knaz,Female,...,2025-01-26,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
3,KB045929,Karma Resorts,Other,Past Guests,Karma Bavaria,PF301535,Herr,Marius,Beinch,Male,...,2025-01-26,2,DIR,WEBSITE,DIGITAL,5,0,0,Email and Phone,NO
4,KK064623,Karma Resorts,Other,Past Guests,Karma Kandara,PF294031,Mr,Kevin,O'Neill,Male,...,2025-01-26,1,DIR,CHAT,DIGITAL,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,KLM007720,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF297476,,Emma Jane,Martin,Unknown,...,2025-02-02,1,OTA,OTA,RETAIL,2,1,0,Email and Phone,NO
59,KLM007662,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF296748,,Russell,Martin,Unknown,...,2025-02-02,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
60,KK066319,Karma Resorts,Other,Past Guests,Karma Kandara,PF297125,Ms,Sala,Buna,Unknown,...,2025-02-02,10,DIGITAL,GHPA,DIGITAL,2,1,0,Email and Phone,NO
61,KLM006630,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF285805,Dr,Fiona,Rae,Unknown,...,2025-02-02,3,DIR,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO


In [18]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Data Setelah Script Past Guest Sebelumnya : "+str(count_data_awal))
print("Jumlah Data Yang Duplicate : "+str(count_duplicate_general))
print("Jumlah Data Setelah Hapus Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Data Yang Termasuk Member: "+str(count_dedup_member))
print("Jumlah Data Setelah Hapus Member : "+str(count_after_dedup_member))
print("Jumlah Data Akhir : "+str(count_data_akhir))

Summary
Jumlah Data Setelah Script Past Guest Sebelumnya : 113
Jumlah Data Yang Duplicate : 42
Jumlah Data Setelah Hapus Duplicate : 71
Jumlah Data Yang Termasuk Member: 8
Jumlah Data Setelah Hapus Member : 63
Jumlah Data Akhir : 63


In [19]:
df_analytic.to_excel(r'C:\Users\fajar\Documents\Python\Data\final_analytics.xlsx', index=False)